<a href="https://colab.research.google.com/github/edipasq/bigdata-challenge/blob/main/kitchenproducts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"


# install findspark using pip
!pip install -q findspark

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-01-25 18:28:18--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-01-25 18:28:18 (5.95 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
#url="https://<bucket name>.s3.amazonaws.com/user_data.csv"
url="https://bucket2-etl-class.s3.us-west-1.amazonaws.com/user_data.csv"
spark.sparkContext.addFile(url)
user_data_df = spark.read.csv(SparkFiles.get("user_data.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
user_data_df.show(5)

+---+----------+---------+-----------+-------------------+--------------+---------+
| id|first_name|last_name|active_user|     street_address|         state| username|
+---+----------+---------+-----------+-------------------+--------------+---------+
|  1|    Cletus|  Lithcow|      false|78309 Riverside Way|      Virginia|ibearham0|
|  2|       Caz|   Felgat|      false|83 Hazelcrest Place|       Alabama| wwaller1|
|  3|     Kerri|  Crowson|      false|     112 Eliot Pass|North Carolina|ichesnut2|
|  4|   Freddie|    Caghy|      false|    15 Merchant Way|      New York|  tsnarr3|
|  5|   Sadella|    Deuss|      false|   079 Acker Avenue|     Tennessee|fwherrit4|
+---+----------+---------+-----------+-------------------+--------------+---------+
only showing top 5 rows



In [6]:
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
w_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Kitchen_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
w_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   37000337|R3DT59XH7HXR9K|B00303FI0G|     529320574|Arthur Court Pape...|         Kitchen|          5|            0|          0|   N|                Y|Beautiful. Looks ...|Beautiful.  Looks...| 2015-08-31|
|         US|   15272914|R1LFS11BNASSU8|B00JCZKZN6|     274237558|Olde Thompson Bav...|         Kitchen|          5|    

In [7]:
from pyspark.sql.functions import col

print(df.count())

In [8]:
print(w_df.count())

4880466


In [9]:
w_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [10]:
print(w_df.count())
w_df = w_df.dropna()
print(w_df.count())
df = w_df.dropDuplicates()
print(w_df.count())

4880466
4879961
4879961


In [11]:
w_df2 = w_df.select(["review_id", "customer_id", "product_id", "product_parent",'review_date'])
w_df2.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3DT59XH7HXR9K|   37000337|B00303FI0G|     529320574| 2015-08-31|
|R1LFS11BNASSU8|   15272914|B00JCZKZN6|     274237558| 2015-08-31|
|R296RT05AG0AF6|   36137863|B00JLIKA5C|     544675303| 2015-08-31|
|R3V37XDZ7ZCI3L|   43311049|B000GBNB8G|     491599489| 2015-08-31|
|R14GU232NQFYX2|   13763148|B00VJ5KX9S|     353790155| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [12]:
from pyspark.sql.functions import *

d=w_df2.select('review_id','customer_id','product_id','product_parent',date_format(col('review_date'),"yyyy-MM-dd").alias('rev').cast("date"))

In [13]:
d.show()

+--------------+-----------+----------+--------------+----------+
|     review_id|customer_id|product_id|product_parent|       rev|
+--------------+-----------+----------+--------------+----------+
|R3DT59XH7HXR9K|   37000337|B00303FI0G|     529320574|2015-08-31|
|R1LFS11BNASSU8|   15272914|B00JCZKZN6|     274237558|2015-08-31|
|R296RT05AG0AF6|   36137863|B00JLIKA5C|     544675303|2015-08-31|
|R3V37XDZ7ZCI3L|   43311049|B000GBNB8G|     491599489|2015-08-31|
|R14GU232NQFYX2|   13763148|B00VJ5KX9S|     353790155|2015-08-31|
| RZQH4V7L2O1PL|   19009420|B00HYB5YY0|     432241873|2015-08-31|
|R1F8JMOSPJ3KO7|   40599388|B000HEBAV2|     584680984|2015-08-31|
|R1ZISGY2BWW4Z5|   22719359|B0012DS4GG|     772637306|2015-08-31|
|R17PW4I3AE5WZW|   47478640|B00FLQ4EE6|     755416578|2015-08-31|
|R3D93G1KTP6A8P|   34195504|B00DBS9OTG|     648762742|2015-08-31|
|R18TQIW1NKPUNU|   19100570|B00AN9UJ68|     495720940|2015-08-31|
|R34KUNL21WU248|   10299811|B00L2P0KNO|      41330497|2015-08-31|
|R2YA1ZA53

In [14]:
d.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- rev: date (nullable = true)



In [15]:
d = d.withColumnRenamed("rev", "review_date")

In [16]:
d.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3DT59XH7HXR9K|   37000337|B00303FI0G|     529320574| 2015-08-31|
|R1LFS11BNASSU8|   15272914|B00JCZKZN6|     274237558| 2015-08-31|
|R296RT05AG0AF6|   36137863|B00JLIKA5C|     544675303| 2015-08-31|
|R3V37XDZ7ZCI3L|   43311049|B000GBNB8G|     491599489| 2015-08-31|
|R14GU232NQFYX2|   13763148|B00VJ5KX9S|     353790155| 2015-08-31|
| RZQH4V7L2O1PL|   19009420|B00HYB5YY0|     432241873| 2015-08-31|
|R1F8JMOSPJ3KO7|   40599388|B000HEBAV2|     584680984| 2015-08-31|
|R1ZISGY2BWW4Z5|   22719359|B0012DS4GG|     772637306| 2015-08-31|
|R17PW4I3AE5WZW|   47478640|B00FLQ4EE6|     755416578| 2015-08-31|
|R3D93G1KTP6A8P|   34195504|B00DBS9OTG|     648762742| 2015-08-31|
|R18TQIW1NKPUNU|   19100570|B00AN9UJ68|     495720940| 2015-08-31|
|R34KUNL21WU248|   10299811|B00L2P0KNO|      41330497| 2015-08

In [17]:
mode="append"
jdbc_url = "jdbc:postgresql://database-1.cz0jfhlrydbo.us-west-1.rds.amazonaws.com:5432/bigdatak"
config = {"user":"postgres",
          "password": "*************",
          "driver":"org.postgresql.Driver"}

In [18]:
d.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [19]:
p_df2 = w_df.select(["product_id", "product_title"])
p_df2.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00303FI0G|Arthur Court Pape...|
|B00JCZKZN6|Olde Thompson Bav...|
|B00JLIKA5C|Progressive Inter...|
|B000GBNB8G|Zyliss Jumbo Garl...|
|B00VJ5KX9S|1 X Premier Pizza...|
+----------+--------------------+
only showing top 5 rows



In [20]:
p_df22 = p_df2.dropDuplicates()

In [21]:
p_df22.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [22]:
cust=w_df.select('customer_id')

In [23]:
print(cust.count())
cust = cust.dropna()
print(cust.count())


4879961
4879961


In [24]:
cust_df = cust.groupBy("customer_id").count()

In [25]:
print(cust_df.count())

2830190


In [26]:
cust_df.show(5)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   23042837|    1|
|   48875707|    2|
|   48088660|    1|
|   20781887|    6|
|    1075003|    1|
+-----------+-----+
only showing top 5 rows



In [27]:
df_index = cust_df.withColumnRenamed("count", "customer_count")

In [28]:
df_index.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   23042837|             1|
|   48875707|             2|
|   48088660|             1|
|   20781887|             6|
|    1075003|             1|
|   28377689|             3|
|    8338749|             1|
|   39271457|             6|
|   38209321|             5|
|   51397605|             1|
|   45337932|             7|
|   12948675|             3|
|   38892468|             1|
|   43582450|             1|
|   13156755|             1|
|   13433330|             1|
|   17067926|             7|
|   13780617|             1|
|   50607818|             1|
|   33437759|             2|
+-----------+--------------+
only showing top 20 rows



In [29]:
df_index.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [ ]:
#df_index = df_index.filter(col("customer_id")  == 10230)
#df_index.sort(col("customer_id")).show(truncate=False)

In [30]:
df_index.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [31]:
vines=w_df.select('review_id','star_rating','helpful_votes','total_votes','vine')

In [32]:
vines.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3DT59XH7HXR9K|          5|            0|          0|   N|
|R1LFS11BNASSU8|          5|            0|          1|   N|
|R296RT05AG0AF6|          5|            0|          0|   N|
|R3V37XDZ7ZCI3L|          5|            0|          1|   N|
|R14GU232NQFYX2|          5|            0|          0|   N|
| RZQH4V7L2O1PL|          1|            1|          1|   N|
|R1F8JMOSPJ3KO7|          5|            0|          0|   N|
|R1ZISGY2BWW4Z5|          5|            0|          0|   N|
|R17PW4I3AE5WZW|          5|            0|          0|   N|
|R3D93G1KTP6A8P|          3|            0|          0|   N|
|R18TQIW1NKPUNU|          5|            0|          0|   N|
|R34KUNL21WU248|          4|            0|          0|   N|
|R2YA1ZA53X12IN|          5|            0|          0|   N|
|R2ZD1IGC9UU55X|          5|            

In [33]:
vines.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [34]:
vines = vines.withColumn("star_rating",col("star_rating").cast("integer"))

In [35]:
vines.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [36]:
vines.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)